In [191]:
import os
import json
import pandas as pd
import traceback

In [192]:
from langchain.chat_models import ChatOpenAI

In [193]:
from dotenv import load_dotenv

load_dotenv() 

False

In [194]:
key=os.getenv("openai")

In [195]:
print(key)

sk-proj-Iq9OS05oYOPNFtzl8j30T3BlbkFJ8RRR2MhEUlf8TXNskQZq


In [196]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.5)

In [197]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021168BFDFA0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021168C04760>, temperature=0.5, openai_api_key='sk-proj-Iq9OS05oYOPNFtzl8j30T3BlbkFJ8RRR2MhEUlf8TXNskQZq', openai_proxy='')

In [198]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [199]:
RESPONSE_JSON={
    "1":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "2":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "3":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    "4":{
        "mcq":"multiple choice questions",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [200]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [201]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [202]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [203]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [204]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [205]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [206]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [207]:
file_path=r"C:\Users\31ash\Desktop\mcq_generator\data.txt"

In [208]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [209]:
print(TEXT)

ChatGPT stands as a pinnacle of technical achievement in the realm of artificial intelligence, boasting an impressive array of capabilities honed through state-of-the-art machine learning techniques. At its core lies a sophisticated neural network architecture, meticulously trained on vast datasets encompassing diverse linguistic patterns and knowledge domains. Through the marvel of deep learning, ChatGPT exhibits a profound understanding of language semantics, syntax, and context, enabling it to comprehend and generate text with remarkable fidelity.

One of its defining features is its ability to engage in contextually relevant conversations, seamlessly tracking the flow of dialogue and adapting responses accordingly. Leveraging techniques such as attention mechanisms and transformer architectures, ChatGPT excels at capturing long-range dependencies within text, facilitating coherent and contextually grounded interactions.

Furthermore, ChatGPT demonstrates impressive language generat

In [210]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [211]:
NUMBER=5 
SUBJECT="TRIGONOMETRY"
TONE="hard"

In [212]:
#setting up Token usage tracking in langchain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
        "text":TEXT,
        "number":NUMBER,
        "subject":SUBJECT,
        "tone":TONE,
        "response_json":json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:ChatGPT stands as a pinnacle of technical achievement in the realm of artificial intelligence, boasting an impressive array of capabilities honed through state-of-the-art machine learning techniques. At its core lies a sophisticated neural network architecture, meticulously trained on vast datasets encompassing diverse linguistic patterns and knowledge domains. Through the marvel of deep learning, ChatGPT exhibits a profound understanding of language semantics, syntax, and context, enabling it to comprehend and generate text with remarkable fidelity.

One of its defining features is its ability to engage in contextually relevant conversations, seamlessly tracking the flow of dialogue and adapting responses accordingly. Leveraging techniques such as attention mechanisms and transformer architectures, ChatGPT excels at capturing long-range dependencies within text, facilitating coh

In [213]:
print(f"total tokens:{cb.total_tokens}")
print(f"Prompt tokens:{cb.prompt_tokens}")
print(f"Completion tokens:{cb.completion_tokens}")
print(f"total cost:{cb.total_cost}")

total tokens:2125
Prompt tokens:1418
Completion tokens:707
total cost:0.003541


In [214]:
quiz=response.get("quiz")

In [215]:
type(quiz)

str

In [216]:
quiz=json.loads(quiz)

In [217]:
quiz

{'1': {'mcq': 'In the realm of artificial intelligence, which technique is predominantly used in ChatGPT for language understanding?',
  'options': {'a': 'Regression analysis',
   'b': 'Decision trees',
   'c': 'Machine learning',
   'd': 'Trigonometry'},
  'correct': 'c'},
 '2': {'mcq': 'What is one of the defining features of ChatGPT mentioned in the text?',
  'options': {'a': 'Ability to bake cookies',
   'b': 'Ability to engage in contextually relevant conversations',
   'c': 'Ability to fly',
   'd': 'Ability to teleport'},
  'correct': 'b'},
 '3': {'mcq': 'Which technique does ChatGPT leverage to capture long-range dependencies within text?',
  'options': {'a': 'Attention mechanisms',
   'b': 'Algebraic equations',
   'c': 'Geometry theorems',
   'd': 'Calculus derivatives'},
  'correct': 'a'},
 '4': {'mcq': 'What is one of the language generation capabilities of ChatGPT mentioned in the text?',
  'options': {'a': 'Playing the piano',
   'b': 'Producing fluent and diverse respons

In [218]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [219]:
quiz_table_data


[{'MCQ': 'In the realm of artificial intelligence, which technique is predominantly used in ChatGPT for language understanding?',
  'Choices': 'a: Regression analysis | b: Decision trees | c: Machine learning | d: Trigonometry',
  'Correct': 'c'},
 {'MCQ': 'What is one of the defining features of ChatGPT mentioned in the text?',
  'Choices': 'a: Ability to bake cookies | b: Ability to engage in contextually relevant conversations | c: Ability to fly | d: Ability to teleport',
  'Correct': 'b'},
 {'MCQ': 'Which technique does ChatGPT leverage to capture long-range dependencies within text?',
  'Choices': 'a: Attention mechanisms | b: Algebraic equations | c: Geometry theorems | d: Calculus derivatives',
  'Correct': 'a'},
 {'MCQ': 'What is one of the language generation capabilities of ChatGPT mentioned in the text?',
  'Choices': 'a: Playing the piano | b: Producing fluent and diverse responses | c: Painting portraits | d: Cooking gourmet meals',
  'Correct': 'b'},
 {'MCQ': 'Which task

In [220]:
quiz=pd.DataFrame(quiz_table_data)

In [221]:
quiz.to_csv("machinelearning.csv",index=False)

In [222]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_15_2024_22_51_53'